In [4]:
import pandas as pd
import json

In [5]:
import json
with open('./harvested_DrugBank_5.1.3.json', 'r') as f:
    drugs = json.load(f)['drugs']

In [6]:
import pandas as pd
db_csv = pd.read_csv('../drugbank_all_drugbank_vocabulary.csv/drugbank vocabulary.csv')


def get_full_name(row):
    if not(pd.isna(row['Synonyms'])):
        return row['Common name'] + ' | ' + row['Synonyms']
    else:
        return row['Common name']
    
db_csv['Full name'] = db_csv.apply(get_full_name, axis=1)
DB_id_2_name = {}
DB_name_2_id = {}
for i, row in db_csv.iterrows():
    DB_id_2_name[row['DrugBank ID']] = row['Full name']
    for name in row['Full name'].split("|"):
        name = name.strip()
        DB_name_2_id[name] = row['DrugBank ID']

In [7]:
texts = []
wanted_keys = ['toxicity', 'metabolism', 'mechanism-of-action', 'description', 'pharmacodynamics', 'indication']
for drug in drugs:
    tmp = drug["drugbank-id"]
    for k,v in drug.items():
        if k in wanted_keys:
            tmp += ' '+v
    texts.append(tmp)

In [28]:
import numpy as np
def add_text_descr(drugbank_id, drugs):
    wanted_keys = ['toxicity', 'metabolism', 'mechanism-of-action', 'description', 'pharmacodynamics', 'indication']
    cur_dic = {}
    for key in wanted_keys:
        cur_dic[key] = np.nan
    for drug in drugs:
        if drug["drugbank-id"] == drugbank_id:
            for key in wanted_keys:
                try:
                    cur_dic[key] = drug[key]
                except KeyError:
                    pass
    return cur_dic
add_text_descr("DB01791", drugs)

{'description': u'Piclamilast (RP-73,401), is a selective PDE4 inhibitor. It is comparable to other PDE4 inhibitors for its anti-inflammatory effects. It has been investigated for its applications to the treatment of conditions such as chronic obstructive pulmonary disease, bronchopulmonary dysplasia and asthma. It is a second generation compound that exhibits structural functionalities of the PDE4 inhibitors cilomilast and roflumilast. The structure for piclamilast was first elucidated in a 1995 European patent application.',
 'indication': nan,
 'mechanism-of-action': nan,
 'metabolism': nan,
 'pharmacodynamics': nan,
 'toxicity': nan}

# LOAD EMBEDDINGS

In [12]:
import pandas as pd
#df = pd.read_csv('./DB_BioBERT_TorchKGE/DB_Bert_Emb.csv')
df = pd.read_csv('./CLEAN_DB_VERSIONS/CLEAN_DB_FOR_GNN/DB_Bert_Emb_INT.csv')
db_ids = sorted(df['DB_ID'].unique().tolist())
db_name_2_id = dict(zip(db_ids,range(len(db_ids))))
print("%d Unique Drugs!" % len(db_ids))

3679 Unique Drugs!


# LOAD MAPPING

In [16]:
df_map = pd.read_csv('./CLEAN_DB_VERSIONS/CLEAN_DB_FOR_GNN/DB_Bert_Emb_INT__MAPPING.csv')
dbid2id = dict(zip(df_map.DB_ID, df_map.INDEX))
id2dbid = dict(zip(df_map.INDEX, df_map.DB_ID))

In [30]:
mapping = {}
for i, row in df.iterrows():
    mapping[id2dbid[row['DB_ID']]] = row[:-1].values
db_name_2_id = {}
for i, v in enumerate(mapping.keys()):
    db_name_2_id[v] = i
id_2_db_name = {}
for db_name, id_ in db_name_2_id.iteritems():
    id_2_db_name[id_] = db_name_2_id

# LOAD INTERACTIONS

In [46]:
type(id2dbid.keys()[0]), type(df_train.h.values[0])

(int, numpy.int64)

In [140]:
import pandas as pd
#df_train = pd.read_csv('./DB_BioBERT_TorchKGE/train_STR__CLEAN_11_12_19.txt', header=None)
df_train = pd.read_csv('./CLEAN_DB_VERSIONS/CLEAN_DB_FOR_GNN/train.txt', header=None, names=['h', 'r', 't'])
df_train['h'] = df_train['h'].map(id2dbid)
df_train['t'] = df_train['t'].map(id2dbid)
df_train.dropna(inplace=True)
print(df_train.shape[0])

df_test = pd.read_csv('./CLEAN_DB_VERSIONS/CLEAN_DB_FOR_GNN/test.txt', header=None, names=['h', 'r', 't'])
df_test['h'] = df_test['h'].map(id2dbid)
df_test['t'] = df_test['t'].map(id2dbid)
df_test.dropna(inplace=True)
print(df_test.shape[0])

290722
1004167


In [141]:
res = []
c_not = 0
embs = []
for db_id in db_name_2_id.keys():
    try:
        cur_dic = {"DB_ID":db_id, "Common Name":DB_id_2_name[db_id].split("|")[0].strip(), "Full Name":DB_id_2_name[db_id], 'Index_ID':db_name_2_id[db_id]}
        descriptions = add_text_descr(db_id, drugs)
        cur_dic.update(descriptions)
        embs.append(mapping[db_id])
        res.append(cur_dic)
    except:
        c_not +=1
print(c_not, c_not/float(len(db_name_2_id)))
df_res = pd.DataFrame(res)
df_embs = pd.DataFrame(np.array(embs))
df_res = pd.concat([df_res, df_embs], axis=1)
#print("Will keep only the rich in information things")
df_res = df_res[df_res.isna().sum(axis=1)<=1]
df_res
#df_res.to_csv("./drug_app/drugs.csv", index=False, encoding='utf8')

(1, 0.0002718129926610492)


,Common Name,DB_ID,Full Name,Index_ID,description,indication,mechanism-of-action,metabolism,pharmacodynamics,toxicity,...,758,759,760,761,762,763,764,765,766,767
1,Mannitol,DB00742,"Mannitol | (2R,3R,4R,5R)-hexane-1,2,3,4,5,6-he...",1886,Mannitol is an osmotic diuretic that is metabo...,Used for the promotion of diuresis before irre...,Mannitol is an osmotic diuretic that is metabo...,"Mannitol is metabolized only slightly, if at a...","Chemically, mannitol is an alcohol and a sugar...",LD<sub>50</sub>=1700 mg/kg (rat oral),...,-0.038572,0.215026,-0.114867,-0.087580,0.057720,-0.210593,0.106146,0.159879,0.025639,0.153972
2,Acenocoumarol,DB01418,Acenocoumarol | 3-(alpha-(4'-Nitrophenyl)-beta...,1769,Acenocoumarol is a coumarin derivative used as...,For the treatment and prevention of thromboemb...,"Acenocoumarol inhibits vitamin K reductase, re...",Extensively metabolized in the liver via oxida...,Acenocoumarol inhibits the reduction of vitami...,The onset and severity of the symptoms are dep...,...,-0.155182,-0.022371,-0.065020,0.052201,0.226994,-0.004546,-0.024331,0.150205,-0.061413,0.078234
4,Benserazide,DB12783,Benserazide | benserazida | Benserazide | bens...,4,When levodopa is used by itself as a therapy f...,The primary therapeutic use for which benseraz...,The combination of levodopa and benserazide is...,Benserazide is hydroxylated to trihydroxybenzy...,When used as a therapy for treating Parkinson'...,Overdosage may lead to cardiovascular side eff...,...,0.009363,0.022773,-0.024202,0.053599,0.059209,-0.094935,0.079551,0.114222,-0.035414,0.098237
7,Sarecycline,DB12035,Sarecycline | Sareciclina | Sarecycline | Saré...,7,Sarecycline is a semi-synthetic derivative of ...,Sarecycline is a tetracycline-class drug indic...,It has been demonstrated that tetracyclines li...,Metabolism of sarecycline by enzymes in human ...,Compared to various examples of first-line tet...,No clinically significant differences in the p...,...,0.048431,0.168045,0.039601,0.160351,0.035760,-0.201687,0.049109,-0.007902,0.147188,0.054707
16,Technetium Tc-99m pyrophosphate,DB09165,Technetium Tc-99m pyrophosphate | Technetium (...,16,A radionuclide imaging agent used primarily in...,For use as a skeletal imaging agent used to de...,It is thought that technetium Tc-99m pyrophosp...,NaN,Technetium Tc-99m pyrophosphate collects in ar...,"Adverse reactions such as flushing, hypotensio...",...,-0.158578,0.108048,-0.032878,-0.058120,0.074874,-0.033773,0.139135,0.303316,0.244350,0.090483
17,Technetium Tc-99m disofenin,DB09164,Technetium Tc-99m disofenin | 99mTc-DISIDA | 9...,148,Technetium Tc-99m disofenin is a radiopharmace...,Technetium Tc99m Disofenin is indicated as a h...,Gallbladder visualization and visualization of...,It undergoes radioactive decay.,Technetium Tc99m is a medical radioisotope tha...,No long-term animal studies have been performe...,...,-0.011036,-0.005964,0.014404,-0.070531,-0.018532,-0.149260,-0.093661,0.152497,0.220477,-0.001374
19,Fosamprenavir,DB01319,Fosamprenavir | FOS-APV | Fosamprenavir,19,"Fosamprenavir is a prodrug of amprenavir, an i...",Indicated in combination with other antiretrov...,Fosamprenavir is a prodrug that is rapidly hyd...,"In the gut epithelium during absorption, fosam...",Fosamprenavir is a pro-drug of the protease in...,NaN,...,-0.040594,-0.042119,0.054049,-0.072531,-0.036478,-0.074674,-0.033787,0.305879,0.150047,0.060772
21,Aluminum sulfate,DB11239,Aluminum sulfate | Aluminium sulfate | Alumini...,21,"Aluminum (Al), also spelled aluminum, chemical...",Solutions containing 5 to 10% aluminum sulfate...,"When used as a deodorant, the volume of sweat ...",NaN,"Aluminum sulfate may be used as a deodorant, a...","Aluminum Sulfate, Hydrated (ACS&FCC): ORAL (LD...",...,0.019860,0.288559,-0.027669,-0.106418,0.065806,0.003080,0.004447,0.039322,0.211023,0.120055
22,Semaglutide,DB13928,Semaglutide | Semaglutide,22,Semaglutide is a once-daily glucagon-like pept...,Semaglutide is indicated to improve glycemic c...,Drugs l

# FInalize the wanted drugs list. We have two limitations:
 - Have enough data for the pre-trained embeddings
 - Be both in train and test data

In [161]:
both_in_train_test = set(df_test.h.unique().tolist() + df_test.t.unique().tolist()).intersection(df_train.h.unique().tolist() + df_train.t.unique().tolist())
print("BOTH IN TRAIN/TEST: %d" % (len(both_in_train_test)))
print("ENOUGH DATA: %d" % (len(df_res.DB_ID.unique())))
final_wanted_DBIDS = both_in_train_test.intersection(df_res.DB_ID.unique()) #both_in_train_test
print("FINAL INTERSECTION: %d" % (len(final_wanted_DBIDS)))

BOTH IN TRAIN/TEST: 2242
ENOUGH DATA: 1914
FINAL INTERSECTION: 1476


# Keep only the features and the DB ID. Also map to integers

In [164]:
df_clean = df_res[[i for i in range(768)] + ['DB_ID']].reset_index(inplace=False, drop=True)
df_clean = df_clean[df_clean.DB_ID.isin(final_wanted_DBIDS)]
df_clean_dbid2id = dict(zip(df_clean.DB_ID.values,range(len(df_clean.DB_ID.values))))
pd.DataFrame({'DB_ID':df_clean_dbid2id.keys(), "INDEX":df_clean_dbid2id.values()}).sort_values('DB_ID').to_csv("./CLEAN_DB_VERSIONS/CLEAN_DB_FOR_GNN_17_02/DB_Bert_Emb_INT__MAPPING.csv", index=False)

In [165]:
df_clean['DB_ID'] = df_clean['DB_ID'].map(df_clean_dbid2id)
assert 0 == df_clean.isna().sum().sum()
df_clean.to_csv("./CLEAN_DB_VERSIONS/CLEAN_DB_FOR_GNN_17_02/DB_Bert_Emb_INT.csv", index=False, header=True)

# FILTER TRAIN/TEST INTERACTIONS TO KEEP ONLY THE UPDATED ONES

In [166]:
df_train_new = df_train[(df_train['h'].isin(final_wanted_DBIDS)) & (df_train['t'].isin(final_wanted_DBIDS))].reset_index(inplace=False, drop=True)
df_test_new = df_test[(df_test['h'].isin(final_wanted_DBIDS)) & (df_test['t'].isin(final_wanted_DBIDS))].reset_index(inplace=False, drop=True)

# MAP BACK TO THE NEW INTEGERS AND SAVE

In [167]:
df_train_new['h'] = df_train_new['h'].map(df_clean_dbid2id)
df_train_new['t'] = df_train_new['t'].map(df_clean_dbid2id)
try:
    assert 0 == df_train_new.isna().sum().sum()
except AssertionError:
    print("SOME DRUGS WHERE FOUND IN THE TRAIN INTERACTIONS BUT DID NOT HAVE EMBEDDINGS. DROPPING THOSE!")
    df_train_new.dropna(inplace=True)
print("NEW TRAIN SHAPE: %d. DROPPED: %d. REDUCTION BY %0.2f %%" % (len(df_train_new), len(df_train) - len(df_train_new), 100*(len(df_train) - len(df_train_new))/float(len(df_train))))
df_train_new.to_csv("./CLEAN_DB_VERSIONS/CLEAN_DB_FOR_GNN_17_02train.txt", header=False, index=False)

NEW TRAIN SHAPE: 170381. DROPPED: 120341. REDUCTION BY 41.39 %


In [168]:
df_test_new['h'] = df_test_new['h'].map(df_clean_dbid2id)
df_test_new['t'] = df_test_new['t'].map(df_clean_dbid2id)
try:
    assert 0 == df_test_new.isna().sum().sum()
except AssertionError:
    print("SOME DRUGS WHERE FOUND IN THE TEST INTERACTIONS BUT DID NOT HAVE EMBEDDINGS. DROPPING THOSE!")
    df_test_new.dropna(inplace=True)
print("NEW TEST SHAPE: %d. DROPPED: %d. REDUCTION BY %0.2f %%" % (len(df_test_new), len(df_test) - len(df_test_new), 100*(len(df_test) - len(df_test_new))/float(len(df_test))))
df_test_new.to_csv("./CLEAN_DB_VERSIONS/CLEAN_DB_FOR_GNN_17_02/test.txt", header=False, index=False)

NEW TEST SHAPE: 304597. DROPPED: 699570. REDUCTION BY 69.67 %
